In [ ]:
!pip install nltk
!pip install beautifulsoup4 requests
!pip install indicnlp
!pip install transformers torch sentence-transformers datasets
import nltk
nltk.download('punkt')

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.2/255.2 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 27.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.5 MB/s eta 0:00:00


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
!pip install nltk
!pip install beautifulsoup4 requests
!pip install indicnlp
!pip install transformers torch sentence-transformers datasets
import nltk
nltk.download('punkt')
!pip install indic-nlp-library
from indicnlp import common

# Set the path to the Indic NLP Resources directory
INDIC_NLP_RESOURCES = "/path_to_resources"

common.set_resources_path(INDIC_NLP_RESOURCES)


import indicnlp
from indicnlp.tokenize import indic_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 67.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.4/587.4 kB 33.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 72.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.1/121.1 kB 8.9 MB/s eta 0:00:00
  Attempting uninstall: docutils
    Found existing installation: docutils 0.18.1
    Uninstalling docutils-0.18.1:
      Successfully uninstalled docutils-0.18.1
  Attempting uninstall: sphinx
    Found existing installation: Sphinx 5.0.2
    Uninstalling Sphinx-5.0.2:
      Successfully uninstalled Sphinx-5.0.2


In [ ]:
import requests
import re
from indicnlp.tokenize import indic_tokenize
from indicnlp.normalize.indic_normalize import IndicNormalizerFactory

# List of common Tamil stop words
tamil_stop_words = [
    "மற்றும்", "இது", "அது", "இவை", "அவை", "என்று", "ஒரு", "இல்லை",
    "உள்ள", "என்ற", "என்ன", "இருந்து", "யார்", "எப்போது", "எங்கே",
    "ஏன்", "எது", "எப்படி", "எத்தனை", "கொண்டு", "போல", "என"
]

# Function to remove suffixes
def remove_suffixes(word):
    suffixes = ["ன்", "ம்", "கள்", "து", "ல்", "டன்", "னின்", "அ", "ஆ", "உம்", "இன்", "கின்", "அவ்"]
    for suffix in suffixes:
        if word.endswith(suffix):
            return word
    return word

# Function to fix endings after suffix removal
def fix_endings(word):
    if word.endswith("ட்டின்"):
        word = word.replace('ட்டின்', "டு")
    elif word.endswith("வின்"):
        word = word.replace("வின்", "")
    elif word.endswith("ங்கள்"):
        word = word[:-1] + "ம்"
    elif word.endswith("ற்"):
        word = word[:-1] + "ற"
    elif word.endswith("வ்"):
        word = word[:-1] + "வ"
    return word

# Complete Tamil stemmer function
def tamil_stemmer(word):
    word = remove_suffixes(word)
    word = fix_endings(word)
    return word

# Function to process tokens with stemming
def process_tokens(words_list):
    tamil_regex = r'[^\u0B80-\u0BFF ]+'  # Tamil Unicode range
    filtered_words = []
    for word in words_list:
        cleaned_word = re.sub(tamil_regex, '', word).strip()
        stemmed_word = tamil_stemmer(cleaned_word)
        if stemmed_word and stemmed_word not in tamil_stop_words:
            filtered_words.append(stemmed_word)
    return filtered_words

# Tokenize and stem the question
def tokenize_question(question):
    factory = IndicNormalizerFactory()
    normalizer = factory.get_normalizer("ta")
    normalized_text = normalizer.normalize(question)
    tokens = list(indic_tokenize.trivial_tokenize(normalized_text, "ta"))
    tokens = [i for i in tokens if i not in tamil_stop_words]
    print(tokens)
    processed_tokens_list = process_tokens(tokens)
    print(processed_tokens_list)
    return processed_tokens_list

# Wikipedia search function
def wikipedia_search(query, language='ta'):
    url = f"https://{language}.wikipedia.org/w/api.php"
    params = {
        'action': 'query',
        'list': 'search',
        'srsearch': query,
        'format': 'json',
        'utf8': 1,
    }
    response = requests.get(url, params=params)
    search_results = response.json()

    return search_results['query']['search'] if 'query' in search_results else []

# Wikipedia article content extraction function
def get_wikipedia_article(title, language='ta'):
    url = f"https://{language}.wikipedia.org/w/api.php"
    params = {
        'action': 'query',
        'prop': 'extracts',
        'exintro': True,
        'explaintext': True,
        'titles': title,
        'format': 'json',
        'utf8': 1,
    }
    response = requests.get(url, params=params)
    pages = response.json()['query']['pages']

    for page_id, page in pages.items():
        if 'extract' in page:
            return page['extract']
    return ""

# Function to answer the question "endraal enna" type questions
def qa(question, language='ta'):
    # Check if the question contains the phrase "என்றால் என்ன"
    match = re.search(r'(.+?)என்றால்\s*என்ன', question)
    if match:
        word_to_define = match.group(1).strip()
        print(f"Searching for: {word_to_define}")

        # Perform Wikipedia search for the extracted word
        search_results = wikipedia_search(word_to_define, language)

        combined_content = ""
        for result in search_results:
            title = result['title']
            content = get_wikipedia_article(title, language)
            combined_content += content + "\n"

        # Return the extracted content
        if combined_content:
            return combined_content
        else:
            return "Sorry, no relevant information was found."
    else:
        return "This question doesn't match the 'endraal enna' pattern."



##Model

In [ ]:
import shutil

# Unzip the file to a specified directory
shutil.unpack_archive('trained_model.zip', './trained_model')

In [ ]:
!unzip trained_model.zip -d ./trained_model


Archive:  trained_model.zip
replace ./trained_model/vocab.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
from transformers import BertForQuestionAnswering, BertTokenizer

# Load the model and tokenizer from the unzipped directory
model = BertForQuestionAnswering.from_pretrained('./trained_model')
tokenizer = BertTokenizer.from_pretrained('./trained_model')


sample questions:

*   "கம்ப்யூட்டர் என்றால் என்ன?"
*   "திருக்குறள் என்றால் என்ன?"
*   "முதலமைச்சர் என்றால் என்ன?"
*   "ஏவூர்தி என்றால் என்ன?"
*   "நீராவி என்றால் என்ன?"
*   "பழங்கள் என்றால் என்ன?"
*   "விடுதலை தினம் என்றால் என்ன?"












In [ ]:
!pip install torch
import torch

In [ ]:
import datetime
start_time = datetime.datetime.now()


test_question = "ஏவூர்தி என்றால் என்ன?"
test_context = qa(test_question)
print(test_context)
inputs = tokenizer(test_question, test_context, truncation=True, padding=True, max_length=512, return_tensors="pt")
with torch.no_grad():
    outputs = model(**inputs)

start_logits = outputs.start_logits
end_logits = outputs.end_logits

# Find the most probable start and end of the answer
start_index = torch.argmax(start_logits)
end_index = torch.argmax(end_logits)

# Convert tokens to answer

answer_tokens = inputs['input_ids'][0][start_index:end_index + 75]
answer = tokenizer.decode(answer_tokens)
# print(answer)
if answer.startswith("<s>"):
    answer = answer[3:]
if len(answer)!=0:
    print("Predicted Answer:", answer)
else:
    print("Sorry:(... Unable to fetch appropriate results!")
end_time = datetime.datetime.now()
print(end_time-start_time)

Searching for: ஏவூர்தி


Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


ஏவூர்தி (Rocket) என்பது ஏவூர்திப் பொறி மூலம் உந்துவிசையைப் பெறும் ஏவுகணை, விண்கலம், விண்ணூர்தி, வானூர்தி போன்றவற்றைக் குறிக்கப் பயன்படுகிறது.  பயன்பாட்டுக்குத் தேவையான எரிபொருள் மற்றும் ஆக்சிகரணி முழுவதும் ஏவூர்தியிலேயே எடுத்துச் செல்லப்படுகிறது.  இதற்கு வளிமண்டலக் காற்று தேவையில்லை.  ஏவூர்திகள் இயற்பியலின் வினை-எதிர்வினை தத்துவத்தில் இயங்குகின்றன.  எரிதல் மூலம் பெறப்பட்ட வெளியேறிகளை அதிக வேகத்தில் பின்புறத்தில் வெளித் தள்ளுவதன் மூலம், ஏவூர்தி பொறிகள் - ஏவூர்திகளை முன் தள்ளுகின்றன.
மற்ற வகை உந்துகைகளுடன் ஒப்பு நோக்குகையில், ஏவூர்திகள் - குறைந்த வேகத்தில் செயல்திறன் அற்றவையாக இருக்கின்றன.  ஏவூர்திகள் குறைந்த எடையும் மிகுந்த திறனும் கொண்டவை.  அவை பெருத்த முடுக்கத்தை அடைவதிலும் மிக உயர்வான திசைவேகங்களை எட்டுவதிலும் மிகுந்த செயல் திறன் கொண்டவையாக இருக்கின்றன.
சோடியம் பைகார்பனேட்டு ஏவூர்தியானது (Sodium bicarbonate rocket), 35 மிமீ ஒளிப்படச் சுருள் டப்பாவையும் சோடியம் பைகார்பனேட்டுடன், ஒரு காடியின் வினையிலிருந்து பெறப்படும் கார்பன் டை ஆக்சைடு வாயுவின் அழுத்தத்தையும் பயன்படுத்திச் செய்யப்படும

In [ ]:
""